In [1]:
# based on <https://github.com/leelabcnbc/thesis-yimeng-v1/blob/master/results_ipynb/yuanyuan_8k_a_3day/transfer_learning_factorized_vgg.ipynb>

In [2]:
from sys import path
from os.path import relpath, realpath, abspath, join, exists
from itertools import product

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py

pd.set_option('display.max_rows', 999)

In [3]:
from torchnetjson.builder import build_net

In [4]:
from thesis_v2 import dir_dict
from thesis_v2.data import load_data_helper
from thesis_v2.training.training_aux import load_training_results
from thesis_v2.training_extra.misc import count_params
from thesis_v2.models.maskcnn_polished.builder import load_modules

load_modules()

In [5]:
folder_to_check = 'scripts/training/yuanyuan_8k_a_3day/maskcnn_polished'
path.insert(0, join(dir_dict['root'], folder_to_check))
from submit import param_iterator_obj
from key_utils import keygen

In [6]:
def load_everything():
    _df_collect = []
    for idx, param in enumerate(param_iterator_obj.generate()):
        if idx % 5 == 0:
            print(idx)
        
        assert len(param) == 16
        assert param['split_seed'] == 'legacy'
        assert param['out_channel'] == 16
        assert param['num_layer'] == 2
        assert param['kernel_size_l1'] == 9
        assert param['pooling_ksize'] == 3
        assert param['pooling_type'] == 'avg'
        key = keygen(**{k: v for k, v in param.items() if k not in {'scale', 'smoothness'}})
        # 10 to go.
        result = load_training_results(key, return_model=False)
        # load twice, first time to get the model.
        result = load_training_results(key, return_model=True, model=build_net(result['config_extra']['model']))
        num_epochs = [len(x) for x in result['stats_all']]
        _df_collect.append(
            {
                'act_fn': param['act_fn'],
                'bn_before_act': param['bn_before_act'],
                'bn_after_fc': param['bn_after_fc'],
                'input_size': param['input_size'],
                'loss_type': param['loss_type'],
                
                # 5 to go
                
                'scale': float(param['scale']),
                # 
                'smoothness': float(param['smoothness']),
                # 
                
                # 1 to go
                
                
                'model_seed': param['model_seed'],
                'corr_test': result['stats_best']['stats']['test']['corr_mean'],
                'max_epoch': max(num_epochs),
                'num_param': count_params(result['model']),
            }
        )
        _df =  pd.DataFrame(_df_collect, columns=['act_fn', 'bn_before_act', 'bn_after_fc',
                                                  'input_size', 'loss_type', 'scale',
                                                  'smoothness', 'model_seed',
                                                  'corr_test', 'num_param',
                                                  'max_epoch',
                                                 ])
        _df = _df.set_index(['act_fn', 'bn_before_act', 'bn_after_fc',
                             'input_size', 'loss_type', 'scale',
                             'smoothness', 'model_seed'])
    return _df

In [7]:
df = load_everything()

0
5


In [8]:
df.unstack('input_size')
# so using 2x downsampled images do improve a bit.
# it's only a bit, compared to transfer learning.
# I think in transfer learning case,
# there is the scale match issue.
# using bigger images match the original training images.

corr_test  \
input_size                                                                     50    
act_fn   bn_before_act bn_after_fc loss_type scale smoothness model_seed             
softplus True          False       poisson   0.01  0.00005    0           0.513621   
                                                              1           0.510794   
                                                              2           0.509790   
                                                              3           0.513501   
                                                              4           0.507801   

                                                                                    \
input_size                                                                     100   
act_fn   bn_before_act bn_after_fc loss_type scale smoothness model_seed             
softplus True          False       poisson   0.01  0.00005    0           0.519386   
                                                              1           0.519152   
                                                              2           0.519938   
                                                              3           0.525551   
                                                              4           0.520909   

                                                                         num_param  \
input_size                                                                     50    
act_fn   bn_before_act bn_after_fc loss_type scale smoothness model_seed             
softplus True          False       poisson   0.01  0.00005    0              20493   
                                                              1              20493   
                                                              2              20493   
                                                              3              20493   
                                                              4              20493   

                                                                                 \
input_size                                                                  100   
act_fn   bn_before_act bn_after_fc loss_type scale smoothness model_seed          
softplus True          False       poisson   0.01  0.00005    0           80928   
                                                              1           80928   
                                                              2           80928   
                                                              3           80928   
                                                              4           80928   

                                                                         max_epoch  \
input_size                                                                     50    
act_fn   bn_before_act bn_after_fc loss_type scale smoothness model_seed             
softplus True          False       poisson   0.01  0.00005    0               9201   
                                                              1               7751   
                                                              2               7951   
                                                              3               7751   
                                                              4               5751   

                                                                                
input_size                                                                 100  
act_fn   bn_before_act bn_after_fc loss_type scale smoothness model_seed        
softplus True          False       poisson   0.01  0.00005    0           4251  
                                                              1           3351  
                                                              2           3951  
                                                              3           4751  
                                                              4           4651